In [3]:
import cv2
import pandas as pd
import numpy as np
from PIL import Image
from os.path import join
from constants import *

In [4]:
cascade_classifier = cv2.CascadeClassifier(CASC_PATH)

In [5]:
def format_image(image, flag=0):
    if len(image.shape) > 2 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE)
    if flag == 0:
        gray_border = np.zeros((150, 150), np.uint8)
        gray_border[:, :] = 200
        gray_border[
            int((150 / 2) - (SIZE_FACE / 2)): int((150 / 2) + (SIZE_FACE / 2)),
            int((150 / 2) - (SIZE_FACE / 2)): int((150 / 2) + (SIZE_FACE / 2))
        ] = image
        image = gray_border
    faces = cascade_classifier.detectMultiScale(
        image,
        scaleFactor=SCALEFACTOR,
        minNeighbors=5
    )

    #  None is we don't found an image
    if not len(faces) > 0:
        return None
    max_area_face = faces[0]
    for face in faces:
        if face[2] * face[3] > max_area_face[2] * max_area_face[3]:
            max_area_face = face
    # Chop image to face
    face = max_area_face
    image = image[face[1]:(face[1] + face[2]), face[0]:(face[0] + face[3])]
    # Resize image to network size
    try:
        image = cv2.resize(image, (SIZE_FACE, SIZE_FACE),
                           interpolation=cv2.INTER_CUBIC) / 255.
    except Exception:
        print("[+] Problem during resize")
        return None
    return image

In [6]:
def emotion_to_vec(x):
    d = np.zeros(len(EMOTIONS))
    d[x] = 1.0
    return d

In [7]:
def flip_image(image):
    return cv2.flip(image, 1)

In [8]:
def data_to_image(data, i):
    data_image = np.fromstring(
        str(data), dtype=np.uint8, sep=' ').reshape((SIZE_FACE, SIZE_FACE))
    data_image = Image.fromarray(data_image).convert('RGB')
    data_image = np.array(data_image)[:, :, ::-1].copy()

#     if you want to save all images
#     cv2.imwrite(SAVE_DIRECTORY + '/images/' + str(i) + '.png', data_image)

    data_image = format_image(data_image)
    return data_image

In [11]:
def get_fer(csv_path,data_type):
    data = pd.read_csv(csv_path)
    labels = []
    images = []
    count = 0
    total_not_valid = 0
    total = data.shape[0]
    if(data_type == 'train') :
        IMG_DIRECTORY = TRAIN_DIRECTORY
    else : 
        IMG_DIRECTORY = TEST_DIRECTORY

    for index, row in data.iterrows():
        emotion = emotion_to_vec(row['emotion'])
        image = data_to_image(row['pixels'], index)

        if image is not None:
            labels.append(emotion)
            images.append(image)

            # if you want to save faces
            real_image = image * 255
            cv2.imwrite(IMG_DIRECTORY +'/'+EMOTIONS[row['emotion']] +'/' + str(index) + '.png', real_image)

            count += 1
        else :
            total_not_valid+=1
            data_image = np.fromstring(str(row['pixels']), dtype=np.uint8, sep=' ').reshape((SIZE_FACE, SIZE_FACE))
            data_image = Image.fromarray(data_image).convert("RGB")
            data_image = np.array(data_image)[:, :, ::-1].copy()
            cv2.imwrite(NOT_VALID_DIRECTORY +'/'+ str(index) + '.png', data_image)
        print("Progress: {}/{} {:.2f}%".format(index, total, index * 100.0 / total))

    print(index)  # 共35587
    print("Total: " + str(len(images)))
    print("Total Not Valid:"+str(total_not_valid))
    np.save(join(SAVE_DIRECTORY,str(SCALEFACTOR) + '_' + SAVE_DATASET_IMAGES_FILENAME), images)
    np.save(join(SAVE_DIRECTORY,str(SCALEFACTOR) + '_' + SAVE_DATASET_LABELS_FILENAME), labels)

In [12]:
get_fer(TRAIN_PATH,'train')

Progress: 0/28709 0.00%
Progress: 1/28709 0.00%
Progress: 2/28709 0.01%
Progress: 3/28709 0.01%
Progress: 4/28709 0.01%
Progress: 5/28709 0.02%
Progress: 6/28709 0.02%
Progress: 7/28709 0.02%
Progress: 8/28709 0.03%
Progress: 9/28709 0.03%
Progress: 10/28709 0.03%
Progress: 11/28709 0.04%
Progress: 12/28709 0.04%
Progress: 13/28709 0.05%
Progress: 14/28709 0.05%
Progress: 15/28709 0.05%
Progress: 16/28709 0.06%
Progress: 17/28709 0.06%
Progress: 18/28709 0.06%
Progress: 19/28709 0.07%
Progress: 20/28709 0.07%
Progress: 21/28709 0.07%
Progress: 22/28709 0.08%
Progress: 23/28709 0.08%
Progress: 24/28709 0.08%
Progress: 25/28709 0.09%
Progress: 26/28709 0.09%
Progress: 27/28709 0.09%
Progress: 28/28709 0.10%
Progress: 29/28709 0.10%
Progress: 30/28709 0.10%
Progress: 31/28709 0.11%
Progress: 32/28709 0.11%
Progress: 33/28709 0.11%
Progress: 34/28709 0.12%
Progress: 35/28709 0.12%
Progress: 36/28709 0.13%
Progress: 37/28709 0.13%
Progress: 38/28709 0.13%
Progress: 39/28709 0.14%
Progress: 

Progress: 327/28709 1.14%
Progress: 328/28709 1.14%
Progress: 329/28709 1.15%
Progress: 330/28709 1.15%
Progress: 331/28709 1.15%
Progress: 332/28709 1.16%
Progress: 333/28709 1.16%
Progress: 334/28709 1.16%
Progress: 335/28709 1.17%
Progress: 336/28709 1.17%
Progress: 337/28709 1.17%
Progress: 338/28709 1.18%
Progress: 339/28709 1.18%
Progress: 340/28709 1.18%
Progress: 341/28709 1.19%
Progress: 342/28709 1.19%
Progress: 343/28709 1.19%
Progress: 344/28709 1.20%
Progress: 345/28709 1.20%
Progress: 346/28709 1.21%
Progress: 347/28709 1.21%
Progress: 348/28709 1.21%
Progress: 349/28709 1.22%
Progress: 350/28709 1.22%
Progress: 351/28709 1.22%
Progress: 352/28709 1.23%
Progress: 353/28709 1.23%
Progress: 354/28709 1.23%
Progress: 355/28709 1.24%
Progress: 356/28709 1.24%
Progress: 357/28709 1.24%
Progress: 358/28709 1.25%
Progress: 359/28709 1.25%
Progress: 360/28709 1.25%
Progress: 361/28709 1.26%
Progress: 362/28709 1.26%
Progress: 363/28709 1.26%
Progress: 364/28709 1.27%
Progress: 36

Progress: 666/28709 2.32%
Progress: 667/28709 2.32%
Progress: 668/28709 2.33%
Progress: 669/28709 2.33%
Progress: 670/28709 2.33%
Progress: 671/28709 2.34%
Progress: 672/28709 2.34%
Progress: 673/28709 2.34%
Progress: 674/28709 2.35%
Progress: 675/28709 2.35%
Progress: 676/28709 2.35%
Progress: 677/28709 2.36%
Progress: 678/28709 2.36%
Progress: 679/28709 2.37%
Progress: 680/28709 2.37%
Progress: 681/28709 2.37%
Progress: 682/28709 2.38%
Progress: 683/28709 2.38%
Progress: 684/28709 2.38%
Progress: 685/28709 2.39%
Progress: 686/28709 2.39%
Progress: 687/28709 2.39%
Progress: 688/28709 2.40%
Progress: 689/28709 2.40%
Progress: 690/28709 2.40%
Progress: 691/28709 2.41%
Progress: 692/28709 2.41%
Progress: 693/28709 2.41%
Progress: 694/28709 2.42%
Progress: 695/28709 2.42%
Progress: 696/28709 2.42%
Progress: 697/28709 2.43%
Progress: 698/28709 2.43%
Progress: 699/28709 2.43%
Progress: 700/28709 2.44%
Progress: 701/28709 2.44%
Progress: 702/28709 2.45%
Progress: 703/28709 2.45%
Progress: 70

Progress: 983/28709 3.42%
Progress: 984/28709 3.43%
Progress: 985/28709 3.43%
Progress: 986/28709 3.43%
Progress: 987/28709 3.44%
Progress: 988/28709 3.44%
Progress: 989/28709 3.44%
Progress: 990/28709 3.45%
Progress: 991/28709 3.45%
Progress: 992/28709 3.46%
Progress: 993/28709 3.46%
Progress: 994/28709 3.46%
Progress: 995/28709 3.47%
Progress: 996/28709 3.47%
Progress: 997/28709 3.47%
Progress: 998/28709 3.48%
Progress: 999/28709 3.48%
Progress: 1000/28709 3.48%
Progress: 1001/28709 3.49%
Progress: 1002/28709 3.49%
Progress: 1003/28709 3.49%
Progress: 1004/28709 3.50%
Progress: 1005/28709 3.50%
Progress: 1006/28709 3.50%
Progress: 1007/28709 3.51%
Progress: 1008/28709 3.51%
Progress: 1009/28709 3.51%
Progress: 1010/28709 3.52%
Progress: 1011/28709 3.52%
Progress: 1012/28709 3.53%
Progress: 1013/28709 3.53%
Progress: 1014/28709 3.53%
Progress: 1015/28709 3.54%
Progress: 1016/28709 3.54%
Progress: 1017/28709 3.54%
Progress: 1018/28709 3.55%
Progress: 1019/28709 3.55%
Progress: 1020/287

Progress: 1304/28709 4.54%
Progress: 1305/28709 4.55%
Progress: 1306/28709 4.55%
Progress: 1307/28709 4.55%
Progress: 1308/28709 4.56%
Progress: 1309/28709 4.56%
Progress: 1310/28709 4.56%
Progress: 1311/28709 4.57%
Progress: 1312/28709 4.57%
Progress: 1313/28709 4.57%
Progress: 1314/28709 4.58%
Progress: 1315/28709 4.58%
Progress: 1316/28709 4.58%
Progress: 1317/28709 4.59%
Progress: 1318/28709 4.59%
Progress: 1319/28709 4.59%
Progress: 1320/28709 4.60%
Progress: 1321/28709 4.60%
Progress: 1322/28709 4.60%
Progress: 1323/28709 4.61%
Progress: 1324/28709 4.61%
Progress: 1325/28709 4.62%
Progress: 1326/28709 4.62%
Progress: 1327/28709 4.62%
Progress: 1328/28709 4.63%
Progress: 1329/28709 4.63%
Progress: 1330/28709 4.63%
Progress: 1331/28709 4.64%
Progress: 1332/28709 4.64%
Progress: 1333/28709 4.64%
Progress: 1334/28709 4.65%
Progress: 1335/28709 4.65%
Progress: 1336/28709 4.65%
Progress: 1337/28709 4.66%
Progress: 1338/28709 4.66%
Progress: 1339/28709 4.66%
Progress: 1340/28709 4.67%
P

...

Progress: 27469/28709 95.68%
Progress: 27470/28709 95.68%
Progress: 27471/28709 95.69%
Progress: 27472/28709 95.69%
Progress: 27473/28709 95.69%
Progress: 27474/28709 95.70%
Progress: 27475/28709 95.70%
Progress: 27476/28709 95.71%
Progress: 27477/28709 95.71%
Progress: 27478/28709 95.71%
Progress: 27479/28709 95.72%
Progress: 27480/28709 95.72%
Progress: 27481/28709 95.72%
Progress: 27482/28709 95.73%
Progress: 27483/28709 95.73%
Progress: 27484/28709 95.73%
Progress: 27485/28709 95.74%
Progress: 27486/28709 95.74%
Progress: 27487/28709 95.74%
Progress: 27488/28709 95.75%
Progress: 27489/28709 95.75%
Progress: 27490/28709 95.75%
Progress: 27491/28709 95.76%
Progress: 27492/28709 95.76%
Progress: 27493/28709 95.76%
Progress: 27494/28709 95.77%
Progress: 27495/28709 95.77%
Progress: 27496/28709 95.77%
Progress: 27497/28709 95.78%
Progress: 27498/28709 95.78%
Progress: 27499/28709 95.79%
Progress: 27500/28709 95.79%
Progress: 27501/28709 95.79%
Progress: 27502/28709 95.80%
Progress: 2750

Progress: 27779/28709 96.76%
Progress: 27780/28709 96.76%
Progress: 27781/28709 96.77%
Progress: 27782/28709 96.77%
Progress: 27783/28709 96.77%
Progress: 27784/28709 96.78%
Progress: 27785/28709 96.78%
Progress: 27786/28709 96.78%
Progress: 27787/28709 96.79%
Progress: 27788/28709 96.79%
Progress: 27789/28709 96.80%
Progress: 27790/28709 96.80%
Progress: 27791/28709 96.80%
Progress: 27792/28709 96.81%
Progress: 27793/28709 96.81%
Progress: 27794/28709 96.81%
Progress: 27795/28709 96.82%
Progress: 27796/28709 96.82%
Progress: 27797/28709 96.82%
Progress: 27798/28709 96.83%
Progress: 27799/28709 96.83%
Progress: 27800/28709 96.83%
Progress: 27801/28709 96.84%
Progress: 27802/28709 96.84%
Progress: 27803/28709 96.84%
Progress: 27804/28709 96.85%
Progress: 27805/28709 96.85%
Progress: 27806/28709 96.85%
Progress: 27807/28709 96.86%
Progress: 27808/28709 96.86%
Progress: 27809/28709 96.87%
Progress: 27810/28709 96.87%
Progress: 27811/28709 96.87%
Progress: 27812/28709 96.88%
Progress: 2781

Progress: 28064/28709 97.75%
Progress: 28065/28709 97.76%
Progress: 28066/28709 97.76%
Progress: 28067/28709 97.76%
Progress: 28068/28709 97.77%
Progress: 28069/28709 97.77%
Progress: 28070/28709 97.77%
Progress: 28071/28709 97.78%
Progress: 28072/28709 97.78%
Progress: 28073/28709 97.78%
Progress: 28074/28709 97.79%
Progress: 28075/28709 97.79%
Progress: 28076/28709 97.80%
Progress: 28077/28709 97.80%
Progress: 28078/28709 97.80%
Progress: 28079/28709 97.81%
Progress: 28080/28709 97.81%
Progress: 28081/28709 97.81%
Progress: 28082/28709 97.82%
Progress: 28083/28709 97.82%
Progress: 28084/28709 97.82%
Progress: 28085/28709 97.83%
Progress: 28086/28709 97.83%
Progress: 28087/28709 97.83%
Progress: 28088/28709 97.84%
Progress: 28089/28709 97.84%
Progress: 28090/28709 97.84%
Progress: 28091/28709 97.85%
Progress: 28092/28709 97.85%
Progress: 28093/28709 97.85%
Progress: 28094/28709 97.86%
Progress: 28095/28709 97.86%
Progress: 28096/28709 97.86%
Progress: 28097/28709 97.87%
Progress: 2809

Progress: 28367/28709 98.81%
Progress: 28368/28709 98.81%
Progress: 28369/28709 98.82%
Progress: 28370/28709 98.82%
Progress: 28371/28709 98.82%
Progress: 28372/28709 98.83%
Progress: 28373/28709 98.83%
Progress: 28374/28709 98.83%
Progress: 28375/28709 98.84%
Progress: 28376/28709 98.84%
Progress: 28377/28709 98.84%
Progress: 28378/28709 98.85%
Progress: 28379/28709 98.85%
Progress: 28380/28709 98.85%
Progress: 28381/28709 98.86%
Progress: 28382/28709 98.86%
Progress: 28383/28709 98.86%
Progress: 28384/28709 98.87%
Progress: 28385/28709 98.87%
Progress: 28386/28709 98.87%
Progress: 28387/28709 98.88%
Progress: 28388/28709 98.88%
Progress: 28389/28709 98.89%
Progress: 28390/28709 98.89%
Progress: 28391/28709 98.89%
Progress: 28392/28709 98.90%
Progress: 28393/28709 98.90%
Progress: 28394/28709 98.90%
Progress: 28395/28709 98.91%
Progress: 28396/28709 98.91%
Progress: 28397/28709 98.91%
Progress: 28398/28709 98.92%
Progress: 28399/28709 98.92%
Progress: 28400/28709 98.92%
Progress: 2840

Progress: 28664/28709 99.84%
Progress: 28665/28709 99.85%
Progress: 28666/28709 99.85%
Progress: 28667/28709 99.85%
Progress: 28668/28709 99.86%
Progress: 28669/28709 99.86%
Progress: 28670/28709 99.86%
Progress: 28671/28709 99.87%
Progress: 28672/28709 99.87%
Progress: 28673/28709 99.87%
Progress: 28674/28709 99.88%
Progress: 28675/28709 99.88%
Progress: 28676/28709 99.89%
Progress: 28677/28709 99.89%
Progress: 28678/28709 99.89%
Progress: 28679/28709 99.90%
Progress: 28680/28709 99.90%
Progress: 28681/28709 99.90%
Progress: 28682/28709 99.91%
Progress: 28683/28709 99.91%
Progress: 28684/28709 99.91%
Progress: 28685/28709 99.92%
Progress: 28686/28709 99.92%
Progress: 28687/28709 99.92%
Progress: 28688/28709 99.93%
Progress: 28689/28709 99.93%
Progress: 28690/28709 99.93%
Progress: 28691/28709 99.94%
Progress: 28692/28709 99.94%
Progress: 28693/28709 99.94%
Progress: 28694/28709 99.95%
Progress: 28695/28709 99.95%
Progress: 28696/28709 99.95%
Progress: 28697/28709 99.96%
Progress: 2869